# Installing necessary dependencies

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.mirror.rafal.ca/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Find Spark and import necessary modules

In [0]:
import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
findspark.init()


from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql import SQLContext
from pyspark.sql import functions as f
from pyspark import SparkConf


sqlContext = SQLContext(spark)

# Creating the DataFrame for the white house data set

In [0]:
 wh_df = spark.read.csv('./100000_White_House_Visitor_Records_Requests.csv.gz', inferSchema=True, header=True)

# Q1A

In [0]:
df_1A = wh_df.groupBy('NAMELAST', 'NAMEFIRST', 'NAMEMID').count().orderBy('count', ascending=False)

In [6]:
q1A_answer = df_1A.take(5)

print('Q1A: 5 most frequent visitors')
print()
df_1A.show(5)

Q1A: 5 most frequent visitors

+--------+---------+-------+-----+
|NAMELAST|NAMEFIRST|NAMEMID|count|
+--------+---------+-------+-----+
|  DEMEKE|    MARIA|   null|   41|
| NEUFELD|     ADAM|   null|   36|
|     KIM|      HUN|      S|   30|
|   MILBY|    SARAH|   null|   25|
|  WIDGER|      ANN|   null|   25|
+--------+---------+-------+-----+
only showing top 5 rows



# Q1B

In [7]:
print('Q1B: Most visited person by each of the 5 most frequent visitors')
print()
for visitor_person in q1A_answer:
  print('Most visited person by', visitor_person.NAMELAST, visitor_person.NAMEFIRST)
  wh_df.filter((f.col('NAMELAST') == visitor_person.NAMELAST) & (f.col('NAMEFIRST') == visitor_person.NAMEFIRST)).groupBy('visitee_namelast', 'visitee_namefirst').count().orderBy('count', ascending=False).show(1)

Q1B: Most visited person by each of the 5 most frequent visitors

Most visited person by DEMEKE MARIA
+----------------+-----------------+-----+
|visitee_namelast|visitee_namefirst|count|
+----------------+-----------------+-----+
|          BUSSOW|             MARK|   21|
+----------------+-----------------+-----+
only showing top 1 row

Most visited person by NEUFELD ADAM
+----------------+-----------------+-----+
|visitee_namelast|visitee_namefirst|count|
+----------------+-----------------+-----+
|          BUSSOW|             MARK|   14|
+----------------+-----------------+-----+
only showing top 1 row

Most visited person by KIM HUN
+----------------+-----------------+-----+
|visitee_namelast|visitee_namefirst|count|
+----------------+-----------------+-----+
|          KUNDRA|            VIVEK|   22|
+----------------+-----------------+-----+
only showing top 1 row

Most visited person by WIDGER ANN
+----------------+-----------------+-----+
|visitee_namelast|visitee_namefirst|c

# Q1C

In [0]:
# It is necesary to change all the null cells, with string 'null' in order to use this DataFrame in the for loop of Q1D
# if not the None values in the list q1C_answer will not match to the null cells in the filter function.
filled_null_df = wh_df.na.fill('null')
df_1C = filled_null_df.groupBy('visitee_namelast', 'visitee_namefirst').count().orderBy('count', ascending=False)

In [10]:
q1C_answer = df_1C.take(6)

print('Q1C: 5 most frequently visited people')
print()
df_1C.show(6)
print('NOTE: We take 6 rows here, since the first two should belong to one visitee')

Q1C: 5 most frequently visited people

+----------------+-----------------+-----+
|visitee_namelast|visitee_namefirst|count|
+----------------+-----------------+-----+
|          OFFICE|         VISITORS|49717|
|          OFFICE|        VISITORS |12063|
|           POTUS|             null| 7381|
|            COLO|          CATRINA| 1714|
|            null|             null| 1688|
|         DOEBLER|              MAX| 1552|
+----------------+-----------------+-----+
only showing top 6 rows

NOTE: We take 6 rows here, since the first two should belong to one visitee


# Q1D



In [11]:
print('Q1D: Each person who visited the most each most visited people')
print()
for visitee_person in q1C_answer:
  # Having checked before that the 'OFFICE' 'VISITORS ' with a space character does not have a person
  # with the most visits to OFFICE, we skip this row and focus only on the first one.
  if visitee_person.visitee_namefirst == 'VISITORS ':
    continue
  else:
    print('Person who visited', visitee_person.visitee_namelast, visitee_person.visitee_namefirst, 'the most.')
    filled_null_df.filter((f.col('visitee_namelast') == visitee_person.visitee_namelast) & (f.col('visitee_namefirst') == visitee_person.visitee_namefirst)).groupBy('NAMELAST', 'NAMEFIRST').count().orderBy('count', ascending=False).show(1)

Q1D: Each person who visited the most each most visited people

Person who visited OFFICE VISITORS the most.
+--------+---------+-----+
|NAMELAST|NAMEFIRST|count|
+--------+---------+-----+
|  COOPER|  HEATHER|   10|
+--------+---------+-----+
only showing top 1 row

Person who visited POTUS null the most.
+--------+---------+-----+
|NAMELAST|NAMEFIRST|count|
+--------+---------+-----+
|   REYES| CAROLINA|    6|
+--------+---------+-----+
only showing top 1 row

Person who visited COLO CATRINA the most.
+--------+---------+-----+
|NAMELAST|NAMEFIRST|count|
+--------+---------+-----+
|   SMITH|   WALTER|    6|
+--------+---------+-----+
only showing top 1 row

Person who visited null null the most.
+---------+---------+-----+
| NAMELAST|NAMEFIRST|count|
+---------+---------+-----+
|BAUSERMAN|  CAMERON|    4|
+---------+---------+-----+
only showing top 1 row

Person who visited DOEBLER MAX the most.
+--------+---------+-----+
|NAMELAST|NAMEFIRST|count|
+--------+---------+-----+
|BOGUSL

# Q1E

In [12]:
print('Q1E: Most frquent visitor-visitee combinations')
visitee_names = wh_df.groupBy('visitee_namelast','visitee_namefirst','NAMELAST','NAMEFIRST','NAMEMID').count().orderBy('count', ascending=False).show(10)

Q1E: Most frquent visitor-visitee combinations
+----------------+-----------------+--------------+---------+-------+-----+
|visitee_namelast|visitee_namefirst|      NAMELAST|NAMEFIRST|NAMEMID|count|
+----------------+-----------------+--------------+---------+-------+-----+
|          KUNDRA|            VIVEK|           KIM|      HUN|      S|   22|
|          BUSSOW|             MARK|        DEMEKE|    MARIA|   null|   21|
|          SAHENE|              JR.|         YODER|     RYAN|   null|   19|
|          KUNDRA|            VIVEK|        HARDIE|     IRIS|      D|   18|
|        WILLIAMS|            DEBRA|        DEMEKE|    MARIA|   null|   17|
|          BUSSOW|             MARK|         MIRZA|     ASMA|   null|   16|
|        WILLIAMS|            DEBRA|         MIRZA|     ASMA|      Y|   15|
|           IRWIN|            JANET|      FONTAINE|  TIMOTHY|      E|   14|
|            WARD|             POPE|         EVANS|    DAVID|      M|   14|
|          FISCUS|           TAYLOR|LEITN